# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vadso,70.0744,29.7487,-1.91,93,75,4.63,NO,1712615574
1,1,kalmunai,7.4167,81.8167,25.11,80,63,0.40,LK,1712615611
2,2,lebu,-37.6167,-73.6500,12.40,81,97,7.57,CL,1712615580
3,3,demir hisar,41.2217,21.2031,10.95,68,4,0.83,MK,1712615612
4,4,port-aux-francais,-49.3500,70.2167,8.18,79,86,15.16,TF,1712615572


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    alpha = 0.5,
    size = "Humidity",
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [70]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[(city_data_df["Max Temp"] <= 29) & 
                             (city_data_df["Max Temp"] > 18) &
                             (city_data_df["Wind Speed"] < 4.5) & 
                             (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,bethel,41.3712,-73.4140,18.05,15,0,2.06,US,1712615544
18,18,arar,30.9753,41.0381,20.01,17,0,2.06,SA,1712615614
62,62,colonia,50.9333,6.9500,19.41,59,0,2.57,DE,1712615545
165,165,darwin,-12.4611,130.8418,26.08,66,0,3.09,AU,1712615453
211,211,al jawf,29.5000,38.7500,19.91,24,0,3.82,SA,1712615646


### Step 3: Create a new DataFrame called `hotel_df`.

In [71]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather.copy()[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,bethel,US,41.3712,-73.4140,15,
18,arar,SA,30.9753,41.0381,17,
62,colonia,DE,50.9333,6.9500,59,
165,darwin,AU,-12.4611,130.8418,66,
211,al jawf,SA,29.5000,38.7500,24,
227,grants pass,US,42.4390,-123.3284,36,
267,marsa alam,EG,25.0757,34.8918,35,
282,san martin,AR,-33.0810,-68.4681,37,
309,palmerston,AU,-12.4860,130.9833,66,
355,marianna,US,30.7744,-85.2269,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [72]:
# Set parameters to search for a hotel
radius = 50000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
arar - nearest hotel: دار أوتيل Dar Otel
colonia - nearest hotel: Wasserturm Hotel Cologne
darwin - nearest hotel: Mantra Pantanas Darwin
al jawf - nearest hotel: No hotel found
grants pass - nearest hotel: The Lodge
marsa alam - nearest hotel: Blue House Hotel
san martin - nearest hotel: Hotel Lacava
palmerston - nearest hotel: Rydges Palmerston
marianna - nearest hotel: Best Value Inn Motel Sandusky
at taj - nearest hotel: No hotel found
westport - nearest hotel: Norwalk Inn and Conference Center
san clemente - nearest hotel: The Volaré
sainte-rose - nearest hotel: Habitation du Compté
jinghong - nearest hotel: โรงแรมสืเบา


,City,Country,Lat,Lng,Humidity,Hotel Name
13,bethel,US,41.3712,-73.4140,15,Hampton Inn Danbury
18,arar,SA,30.9753,41.0381,17,دار أوتيل Dar Otel
62,colonia,DE,50.9333,6.9500,59,Wasserturm Hotel Cologne
165,darwin,AU,-12.4611,130.8418,66,Mantra Pantanas Darwin
211,al jawf,SA,29.5000,38.7500,24,No hotel found
227,grants pass,US,42.4390,-123.3284,36,The Lodge
267,marsa alam,EG,25.0757,34.8918,35,Blue House Hotel
282,san martin,AR,-33.0810,-68.4681,37,Hotel Lacava
309,palmerston,AU,-12.4860,130.9833,66,Rydges Palmerston
355,marianna,US,30.7744,-85.2269,47,Best Value Inn Motel Sandusky


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [76]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    alpha = 0.5,
    size = "Humidity",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)